In [ ]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install scikit-learn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import cuml
import cudf
from cuml.model_selection._split import train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
file_list = ["garmin_edge_820/3993730634_ACTIVITY_data.csv",
             "garmin_edge_820/4557226804_ACTIVITY_data.csv",
             "garmin_edge_820/4593452980_ACTIVITY_data.csv",
             "garmin_edge_820/5191513011_ACTIVITY_data.csv",
]
combined_df = pd.concat([pd.read_csv(file, sep=";") for file in file_list], ignore_index=True)

In [ ]:
def convert_brackets(string):
    return string.replace('[', '(').replace(']', ')')

combined_df.columns = [convert_brackets(col) for col in combined_df.columns]

In [ ]:
combined_df['timestamp(s)'] = combined_df['timestamp(s)'] + 631065600
combined_df['time'] = pd.to_datetime(combined_df.pop('timestamp(s)'), unit='s')
combined_df.set_index("time", inplace=True)

In [ ]:
combined_df = combined_df[combined_df['speed(m/s)'] != 0]
combined_df = combined_df.dropna(subset=['speed(m/s)'])

In [ ]:
combined_df['time_since_start(s)'] = combined_df.groupby(pd.Grouper(freq='D')).cumcount() + 1

In [ ]:
hr_zones = [(0, 128), (129, 146), (147, 156), (157, 165),(166, 174), (175, 179), (180, float('inf'))]
power_zones = [(0, 157), (158, 186), (187, 200), (201, 218),(219, 247), (248, 287), (288, float('inf'))]

def get_zone(rate, zones):
    for zone, (lower, upper) in enumerate(zones, start=1):
        if lower <= rate <= upper:
            return zone
        
combined_df['hr_zone'] = combined_df['heart_rate(bpm)'].apply(get_zone, zones=hr_zones)
combined_df['pwr_zone'] = combined_df['power(watts)'].apply(get_zone, zones=power_zones)

In [ ]:
combined_df['altitude_diff(m)'] = combined_df['altitude(m)'] - combined_df['altitude(m)'].shift(1)
combined_df['distance_diff(m)'] = combined_df['distance(m)'] - combined_df['distance(m)'].shift(1)
combined_df[['altitude_diff(m)', 'distance_diff(m)']] = combined_df[['altitude_diff(m)', 'distance_diff(m)']].fillna(0)
combined_df['slope_percent'] = np.where(combined_df['distance_diff(m)'] == 0, 0, combined_df['altitude_diff(m)'] / combined_df['distance_diff(m)'] * 100)

In [ ]:
combined_df.drop(combined_df[combined_df['heart_rate(bpm)'] < 80].index, inplace=True)

In [ ]:
window_size = 3 
combined_df['avg_power(watts)'] = combined_df['power(watts)'].rolling(window=int(window_size), center=True).mean()
combined_df['avg_power(watts)'] = combined_df['avg_power(watts)'].dropna()

In [ ]:
combined_df['power_left(watts)'] = combined_df['left_right_balance'] - 128
combined_df['power_right(watts)'] = 100 - combined_df['power_left(watts)']

In [ ]:
# Pulizia dei dati
details = details.drop(['left_power_phase[degrees]',
                        'left_power_phase_peak[degrees]',
                        'right_power_phase[degrees]',
                        'right_power_phase_peak[degrees]',
                        'left_right_balance'], axis=1)




# Calcola la differenza di altitudine tra le righe adiacenti
details['altitude_diff'] = details['altitude(m)'] - \
    details['altitude(m)'].shift(1)
details['distance_diff'] = details['distance(m)'] - \
    details['distance(m)'].shift(1)
details[['altitude_diff', 'distance_diff']] = details[[
    'altitude_diff', 'distance_diff']].fillna(0)
# Calcola la percentuale di pendenza
details['slope_percent'] = np.where(
    details['distance_diff'] == 0, 0, details['altitude_diff'] / details['distance_diff'] * 100)
details['speed(m/s)'] = details['speed(m/s)'].fillna(details['speed(m/s)'].mean())

In [ ]:
details = cudf.DataFrame(details)
X = details.drop('heart_rate(bpm)', axis=1)
y = details['heart_rate(bpm)']
X_train, X_test, y_train, y_test = cuml.model_selection.train_test_split(X, y, test_size=1/3, train_size=0.75,  random_state=42)

In [ ]:
X_cudf_train = cudf.DataFrame.from_pandas(X_train)
X_cudf_test = cudf.DataFrame.from_pandas(X_test)

y_cudf_train = cudf.Series(y_train.values)

In [ ]:
lr = cuml.LinearRegression(algorithm='svd', fit_intercept=True, normalize=True)
lr.fit(X_cudf_train,y_cudf_train)